# The forward and backward passes

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Loading data
Lets begin by loading our data

In [3]:
#export
from exp.nb_01 import *

def load_data(folder):
    dataList = []
    labelList = []
    for num in range(10):
        data_path = (path/folder/f'{num}').ls().sorted() #getting path
        
        stackedData = torch.stack([tensor(Image.open(o)) for o in data_path]) #Open each image and stack them
        stackedData = stackedData.float()/255.0 #squishing between 0-1
        
        dataList.append(stackedData) #adding to dataList
        labelList.extend([num]*len(data_path))#extending labelList
    
    #Convert so that each image data is in each row
    train_x = torch.cat(dataList).view(-1, 28*28) 
    train_y = tensor(labelList)
    
    return train_x, train_y

def normalize(x, m, s): return (x-m)/s #x=tensor, m=mean, s=stdev

In [4]:
path = untar_data(URLs.MNIST)
path

x_train, y_train = load_data("training")
x_valid, y_valid = load_data("testing")

### Normalization
We want our mean and std to be 0,1 respectivly.

In [5]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1307), tensor(0.3081))

> We want our mean to be near 0, and our std to be 1: It's not, so lets normalize

In [6]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

> Notice we use training, not validation mean for validation set

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-5.7298e-09), tensor(1.0000))

> Much better

In [8]:
#Same for valid
x_valid.mean(),x_valid.std()

(tensor(0.0060), tensor(1.0077))

In [25]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [11]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(60000, 784, tensor(10))

## Foundations version
Lets create our ml model from scratch.

### Initialize parameters

In [12]:
# num hidden
nh = 50

In [27]:
# standard xavier init
                #784,50         
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh) #output is 1 because we will be doing a binary classification for now
b2 = torch.zeros(1)

In [33]:
w1.mean(), w1.std()

(tensor(0.0002), tensor(0.0358))

In [29]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

### Linear

In [34]:
def lin(x, w, b): return x@w + b

In [16]:
t = lin(x_valid, w1, b1)

In [17]:
t.mean(),t.std()

(tensor(0.1286), tensor(1.0396))

> Nice mean and std is 0,1

### Activation func

In [18]:
def relu(x): return x.clamp_min(0.) #activation func

In [19]:
t = relu(lin(x_valid, w1, b1))

In [20]:
t.mean(),t.std()

(tensor(0.4762), tensor(0.6551))

> Notice that our mean and std is not 0,1: But actually it really should be this!

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [21]:
# Lets adjust our weight to work with ReLU
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [22]:
w1.mean(),w1.std() 

(tensor(-1.4533e-05), tensor(0.0510))

In [23]:
t = relu(lin(x_valid, w1, b1)) #after we go through ReLU
t.mean(),t.std()

(tensor(0.5088), tensor(0.8018))

> Notice the mean is not 0, instead is .5: Remember we are using ReLU so half our data has been removed. 

In [24]:
#export
from torch.nn import init

In [25]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

> init.kaiming_normal_ = torch.randn(m,nh)*math.sqrt(2/m)

In [26]:
init.kaiming_normal_??

In [27]:
w1.mean(),w1.std() 

(tensor(0.0004), tensor(0.0505))

In [28]:
t.mean(),t.std() #Similer to before

(tensor(0.5449), tensor(0.8190))

In [ ]:
w1.shape

torch.Size([784, 50])

In [30]:
import torch.nn

In [31]:
torch.nn.Linear(m,nh).weight.shape 

torch.Size([50, 784])

> Notice that m and nh are switched

In [ ]:
torch.nn.Linear.forward??

In [ ]:
torch.nn.functional.linear?? #Does matrix and transposes

In [ ]:
torch.nn.Conv2d??

In [ ]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [32]:
def relu(x): return x.clamp_min(0.) - 0.5

> Why dont we adjust our activation func? Our hope is that this brings our mean of .5 to 0

In [34]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.1114), tensor(0.8534))

> Awesome, it did improve it!

### Model

In [35]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [36]:
%timeit -n 10 _=model(x_valid)

5.1 ms ± 476 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [38]:
model(x_valid).shape

torch.Size([10000, 1])

In [39]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [40]:
y_train,y_valid = y_train.float(),y_valid.float()

### Forward pass
Lets calculate our predictions

In [42]:
preds = model(x_train)

In [43]:
preds.shape

torch.Size([60000, 1])

In [44]:
mse(preds, y_train)

tensor(25.6164)

### Gradients and backward pass

In [45]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [46]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [47]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [48]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [49]:
forward_and_backward(x_train, y_train)

In [50]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [51]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [52]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [53]:
loss = forward(xt2, y_train)

In [55]:
loss.backward()

> here we call backward, this does exactly the same backward pass that we did in our forward_and_backward func

In [56]:
#Lets compare our results
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

In [61]:
w2g[:5], w22.grad[:5]

(tensor([[3.3452],
         [3.6163],
         [1.3053],
         [2.1887],
         [3.0934]]),
 tensor([[3.3452],
         [3.6163],
         [1.3053],
         [2.1887],
         [3.0934]]))

## Refactoring our model

In [63]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [64]:
class Lin():
    def __init__(self, w, b): 
        self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [65]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [66]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x) #Forward pass
        return self.loss(x, targ) #Return loss
    
    def backward(self):
        self.loss.backward() 
        for l in reversed(self.layers): l.backward()

In [75]:
w1.g,b1.g,w2.g,b2.g = [None]*4

In [76]:
model = Model(w1, b1, w2, b2)

In [79]:
%time loss = model(x_train, y_train)

CPU times: user 184 ms, sys: 68.6 ms, total: 253 ms
Wall time: 36.9 ms


In [80]:
%time model.backward()

CPU times: user 7.28 s, sys: 8.34 s, total: 15.6 s
Wall time: 2.51 s


In [81]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Further Refactoring - Module.forward()

In [84]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [85]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [86]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [87]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [88]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [89]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [90]:
%time loss = model(x_train, y_train)

CPU times: user 158 ms, sys: 66.8 ms, total: 225 ms
Wall time: 28.8 ms


In [91]:
%time model.backward()

CPU times: user 337 ms, sys: 444 ms, total: 781 ms
Wall time: 99.3 ms


In [92]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [93]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [94]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [95]:
%time loss = model(x_train, y_train)

CPU times: user 173 ms, sys: 42.8 ms, total: 216 ms
Wall time: 27.3 ms


In [96]:
%time model.backward()

CPU times: user 305 ms, sys: 417 ms, total: 722 ms
Wall time: 91.4 ms


In [97]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## nn.Linear and nn.Module
Above we have basically create nn.Linear and nn.Module: So now lets use them!

In [98]:
#export
from torch import nn

In [100]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [101]:
model = Model(m, nh, 1)

In [102]:
%time loss = model(x_train, y_train)

CPU times: user 157 ms, sys: 68.4 ms, total: 226 ms
Wall time: 31 ms


In [103]:
%time loss.backward()

CPU times: user 284 ms, sys: 33 ms, total: 317 ms
Wall time: 40.3 ms


## Export

In [104]:
!./notebook2script.py 02_fully_connected.ipynb

/bin/bash: ./notebook2script.py: No such file or directory
